<h1> Applied Data Science Capstone</h1>

<h1>Capstone Project - The Battle of Neighborhoods</h1>

<h2>1. Obtain the data </h2>
<h3>1.1 Data used to define the neighborhoods from Wikipedia </h3>

<h3> Preparation:</h3>
<h4>Do the software installs for Part 1 - only once per launch of notebook</h4>

In [1]:
!conda install -c conda-forge beautifulsoup4 --yes
print("INSTALLED BEAUTIFULSOUP4")
!conda install -c conda-forge lxml --yes
print("INSTALLED LXML")
!conda install -c conda-forge requests --yes
print("INSTALLED REQUESTS")

Solving environment: ...working... done

# All requested packages already installed.

INSTALLED BEAUTIFULSOUP4
Solving environment: ...working... done

# All requested packages already installed.

INSTALLED LXML
Solving environment: ...working... done

# All requested packages already installed.

INSTALLED REQUESTS


<h3>Do all the imports - with launch or when you have to restart the python kernel</h3>

In [2]:
from bs4 import BeautifulSoup
import requests

from lxml import html

import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

print('IMPORTS DONE')


IMPORTS DONE


<h3>Import the Canadian Post codes table from the Wikipedia webpage utilising BeautifulSoup</h3>

In [3]:

with open('C:\DATA\AI\cournsera\List of postal codes of Canada Wikipedia.html',encoding="utf-8") as f:
    source = f.read()


print("SOURCE")

soup = BeautifulSoup(source,'lxml')

print("SOUP")
#print(soup)
          
post_codes = soup.find('table',class_="wikitable sortable jquery-tablesorter")

print("POST_CODES")
print(post_codes)

print('FOUND THE POST CODE TABLE ON WIKIPEDIA')

SOURCE
SOUP
POST_CODES
<table class="wikitable sortable jquery-tablesorter">
<thead><tr>
<th class="headerSort" role="columnheader button" tabindex="0" title="Sort ascending">Postcode</th>
<th class="headerSort" role="columnheader button" tabindex="0" title="Sort ascending">Borough</th>
<th class="headerSort" role="columnheader button" tabindex="0" title="Sort ascending">Neighbourhood
</th></tr></thead><tbody>
<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A</td>
<td><a href="https://en.wikipedia.org/wiki/North_York" title="North York">North York</a></td>
<td><a href="https://en.wikipedia.org/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>
<tr>
<td>M4A</td>
<td><a href="https://en.wikipedia.org/wiki/North_York" title="North York">North York</a></td>
<td><a href="https://en.wikipedia.org/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td></tr>
<tr>
<td>M5

<h4>Use BeautifulSoup find_all to extract the rows from the Canadian Postal code table</h4>

In [4]:
table_rows = post_codes.find_all('tr')

<h3>Prepare the empty Pandas Dataframe</h3>

In [5]:
# define the dataframe columns
column_names = ['PostalCode','Borough', 'Neighborhood'] 

# instantiate the dataframe
pd_post_codes = pd.DataFrame(columns=column_names)

print(pd_post_codes.columns)
pd_post_codes


Index(['PostalCode', 'Borough', 'Neighborhood'], dtype='object')


,PostalCode,Borough,Neighborhood


<h3>Populate the Pandas Dataframe</h3>

#### Loop through the table rows and add it to the pd_post_codes dataframe


In [6]:
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text for tr in td] # contains a list
    if len(row)> 0:
        #Only non-empty rows will be added
        row_postalcode = row[0]
        if row[1] !=  'Not assigned': 
            # borough that is assigned
            row_borough = row[1]
            row_neighborhood = row[2] 
            #  - w.r.t. the "\n" - I am using it to put commmas in between the Neighborhoods. 
            if row_neighborhood == 'Not assigned\n':
                row_neighborhood = row_borough
            pd_post_codes = pd_post_codes.append({'PostalCode':row_postalcode,'Borough':row_borough, 'Neighborhood':row_neighborhood},ignore_index=True)

print('The rows should now be in pd_post_codes - albeit with possibly more than one row per post code') 



The rows should now be in pd_post_codes - albeit with possibly more than one row per post code


<h3>Replace the "\n" in the Neighborhood column with a comma so that there will be a comma between the neighbourhoods once concatenated.

In [8]:
pd_post_codes['Neighborhood'] = pd_post_codes['Neighborhood'].str.replace('\n',',') 

<h3>Combine neighborhoods with the same borough

In [9]:
pd_post_codes = pd_post_codes.groupby(['PostalCode','Borough'],as_index=False)['Neighborhood'].sum()

<h4> Remove the last character - which is the comma of the rightmost neighborhood in the list</h4>

In [11]:
pd_post_codes['Neighborhood']= pd_post_codes['Neighborhood'].str[:-1]

In [12]:
pd_post_codes.shape

(103, 3)

<h3> Read in the csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data</h3>
    

In [13]:
Postal_codes_coordinates='http://cocl.us/Geospatial_data'
    
pd_Postal_codes_coordinates = pd.read_csv(Postal_codes_coordinates)

<h3> Rename the 'Postal Codes'- column in order for the Merge to work. (I have tried it without renaming.)

In [14]:
pd_Postal_codes_coordinates.rename(columns={'Postal Code':'PostalCode'},inplace=True)


<h3>Merge the two dataframes using the PostalCode column to join.</h3>

In [15]:

pd_Postal_codes_part2 =pd.merge(pd_post_codes,pd_Postal_codes_coordinates[['PostalCode','Latitude','Longitude']],
left_on='PostalCode',
right_on='PostalCode',
how='outer')

pd_Postal_codes_part2.shape

(103, 5)

<h3>1.2. VENUE data from FOURSQUARE.com</h3>

<h3>Preparation</h3>

<h3>Do the installs</h3>


In [16]:
!conda install -c conda-forge geopy --yes 
print('INSTALLED GEOPY')
!conda install -c conda-forge folium=0.5.0 --yes 
print('INSTALLED FOLIUM')
print('INSTALLATION DONE.')


Solving environment: ...working... done

# All requested packages already installed.

INSTALLED GEOPY
Solving environment: ...working... done

# All requested packages already installed.

INSTALLED FOLIUM
INSTALLATION DONE.


<h3> Do the imports</h3>

In [17]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from pandas.io.json import json_normalize

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import json # library to handle JSON files


import folium # map rendering library

print('LIBRARIES IMPORTED.')

LIBRARIES IMPORTED.


A Pandas DataFrame has been created for Toronto that is similar to the one for New York in Part2.

In [19]:
pd_Postal_codes_part2.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malver",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Unio",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hil",43.763573,-79.188711
3,M1G,Scarborough,Wobur,43.770992,-79.216917
4,M1H,Scarborough,Cedarbra,43.773136,-79.239476


But we only want to work with Toronto.

In [20]:
pd_Postal_codes_Toronto=pd_Postal_codes_part2[pd_Postal_codes_part2['Borough'].str.contains('Toronto')]


In [22]:
print('Now we are only working with {} boroughs in Toronto.'.format(pd_Postal_codes_Toronto.shape[0]))
print('Here are the first 5 rows')
pd_Postal_codes_Toronto.head()


Now we are only working with 39 boroughs in Toronto.
Here are the first 5 rows


,PostalCode,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,The Beache,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West,Riverdal",43.679557,-79.352188
42,M4L,East Toronto,"The Beaches West,India Bazaa",43.668999,-79.315572
43,M4M,East Toronto,Studio Distric,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Par,43.728020,-79.388790


#### Use geopy library to get the latitude and longitude values of Toronto.

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>ny_explorer</em>, as shown below.

In [23]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


#### Create a map of Toronto with neighborhoods superimposed on top.

In [24]:
# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(pd_Postal_codes_Toronto['Latitude'], pd_Postal_codes_Toronto['Longitude'], pd_Postal_codes_Toronto['Borough'], pd_Postal_codes_Toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  


map_Toronto

Please feel free to zoom in where the action is using the "+" in the tot left corner.

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

#### Define Foursquare Credentials and Version - Hidden

In [25]:
# The code was removed by Watson Studio for sharing.

### Obtain the Venue data from FOURSQUARE.com

#### Getting all the venues, using the latitude and Longitude of the post code as that of the Neighborhood

In [28]:
def getNearbyVenues(names, latitudes, longitudes, radius=500,LIMIT=100):
    
    # instantiate the dataframe
    
    column_names = ['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude', 
                 'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category']
    nearby_venues = pd.DataFrame(columns=column_names)
    
    # for each Post Code, add all the venues to nearby_venues
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        #make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # Get to the relevant part in the JSON-data returned from FOURSQUARE.com
        venues = results
                
        N = len(venues)
        for i in range(N):
            icon_prefix = venues[i]['venue']['categories'][0]['icon']['prefix']
            row_category_group=icon_prefix.split('/')[5]
            if row_category_group == 'food':
                row_category_name = venues[i]['venue']['categories'][0]['name']
                row_establishment = venues[i]['venue']['name']
                row_establishment_latitude = venues[i]['venue']['location']['lat']
                row_establishment_longitude = venues[0]['venue']['location']['lng']
                nearby_venues = nearby_venues.append({'Neighborhood':name, 'Neighborhood Latitude':lat,
                                              'Neighborhood Longitude':lng,
                                              'Venue':row_establishment, 
                                              'Venue Latitude':row_establishment_latitude, 
                                              'Venue Longitude':row_establishment_longitude,
                                              'Venue Category':row_category_name},ignore_index=True)
    
    return(nearby_venues)

#### Execution of the above function on each neighborhood and create a new dataframe called *Toronto_venues*.

In [29]:

toronto_venues = getNearbyVenues(names=pd_Postal_codes_Toronto['Neighborhood'],
                                   latitudes=pd_Postal_codes_Toronto['Latitude'],
                                   longitudes=pd_Postal_codes_Toronto['Longitude']
                                  )



The Beache
The Danforth West,Riverdal
The Beaches West,India Bazaa
Studio Distric
Lawrence Par
Davisville Nort
North Toronto Wes
Davisvill
Moore Park,Summerhill Eas
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill Wes
Rosedal
Cabbagetown,St. James Tow
Church and Wellesle
Harbourfron
Ryerson,Garden Distric
St. James Tow
Berczy Par
Central Bay Stree
Adelaide,King,Richmon
Harbourfront East,Toronto Islands,Union Statio
Design Exchange,Toronto Dominion Centr
Commerce Court,Victoria Hote
Roselaw
Forest Hill North,Forest Hill Wes
The Annex,North Midtown,Yorkvill
Harbord,University of Toront
Chinatown,Grange Park,Kensington Marke
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagar
Stn A PO Boxes 25 The Esplanad
First Canadian Place,Underground cit
Christi
Dovercourt Village,Dufferi
Little Portugal,Trinit
Brockton,Exhibition Place,Parkdale Villag
High Park,The Junction Sout
Parkdale,Roncesvalle
Runnymede,Swanse
Queen's Par
Business Reply M

Let's check how many venues were returned for each neighborhood

In [30]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmon",60,60,60,60,60,60
Berczy Par,26,26,26,26,26,26
"Brockton,Exhibition Place,Parkdale Villag",11,11,11,11,11,11
Business Reply Mail Processing Centre 969 Easter,5,5,5,5,5,5
"CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagar",1,1,1,1,1,1
"Cabbagetown,St. James Tow",28,28,28,28,28,28
Central Bay Stree,62,62,62,62,62,62
"Chinatown,Grange Park,Kensington Marke",53,53,53,53,53,53
Christi,7,7,7,7,7,7


### Unique categories of food places  and it counts

In [32]:
tv_columns = toronto_venues.groupby(['Venue Category'])['Venue'].count()
tv_columns

Venue Category
Afghan Restaurant                    1
American Restaurant                 23
Asian Restaurant                    11
BBQ Joint                            5
Bagel Shop                           3
Bakery                              33
Belgian Restaurant                   1
Bistro                               4
Brazilian Restaurant                 2
Breakfast Spot                      17
Brewery                             12
Bubble Tea Shop                      9
Burger Joint                        13
Burrito Place                        9
Café                                89
Cajun / Creole Restaurant            1
Caribbean Restaurant                 4
Chinese Restaurant                  10
Coffee Shop                        157
Colombian Restaurant                 2
Comfort Food Restaurant              7
Creperie                             7
Cuban Restaurant                     2
Cupcake Shop                         1
Deli / Bodega                       12
Dessert Sh

<h3>Neighborhoods with count of respective food categories</h3>

In [33]:
toronto_venues.groupby(['Neighborhood','Venue Category']).count()

Neighborhood Latitude  \
Neighborhood                                       Venue Category                                           
Adelaide,King,Richmon                              American Restaurant                                  2   
                                                   Asian Restaurant                                     2   
                                                   Bakery                                               3   
                                                   Brazilian Restaurant                                 1   
                                                   Breakfast Spot                                       2   
                                                   Burger Joint                                         1   
                                                   Burrito Place                                        1   
                                                   Café                                                 5   
                                                   Coffee Shop                                          7   
                                                   Colombian Restaurant                                 1   
                                                   Deli / Bodega                                        1   
                                                   Fast Food Restaurant                                 1   
                                                   Gastropub                                            2   
                                                   Gluten-free Restaurant                               1   
                                                   Ice Cream Shop                                       1   
                                                   Japanese Restaurant                                  1   
                                                   Juice Bar                                            1   
                                                   Latin American Restaurant                            1   
                                                   Mediterranean Restaurant                             1   
                                                   New American Restaurant                              1   
                                                   Noodle House                                         1   
                                                   Pizza Place                                          2   
                                                   Poke Place                                           1   
                                                   Ramen Restaurant                                     1   
                                                   Restaurant                                           6   
                                                   Salad Place                                          2   
                                                   Sandwich Place                                       1   
                                                   Seafood Restaurant                                   1   
                                                   Steakhouse                                           2   
                                                   Sushi Restaurant                                     2   
                                                   Thai Restaurant                                      3   
                                                   Vegetarian / Vegan Restaurant                        2   
Berczy Par                                         BBQ Joint                                            1   
                                                   Bagel Shop                                           1   
                                                   Bakery                                               2   
                                                   

In [34]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 81 uniques categories.


<a id='item3'></a>

## 2. Analyze Each Neighborhood

In [35]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])

toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,Afghan Restaurant,American Restaurant,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Belgian Restaurant,Bistro,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,Colombian Restaurant,Comfort Food Restaurant,Creperie,Cuban Restaurant,Cupcake Shop,Deli / Bodega,Dessert Shop,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Empanada Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Food,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Gastropub,German Restaurant,Gluten-free Restaurant,Greek Restaurant,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Latin American Restaurant,Mac & Cheese Joint,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Molecular Gastronomy Restaurant,New American Restaurant,Noodle House,Pizza Place,Poke Place,Polish Restaurant,Poutine Place,Ramen Restaurant,Restaurant,Salad Place,Sandwich Place,Seafood Restaurant,Snack Place,Steakhouse,Sushi Restaurant,Taiwanese Restaurant,Tea Room,Thai Restaurant,Theme Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wings Joint
0,"The Danforth West,Riverdal",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"The Danforth West,Riverdal",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"The Danforth West,Riverdal",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"The Danforth West,Riverdal",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"The Danforth West,Riverdal",0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


And let's examine the new dataframe size.

In [36]:
toronto_onehot.shape

(988, 82)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [37]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Afghan Restaurant,American Restaurant,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Belgian Restaurant,Bistro,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,Colombian Restaurant,Comfort Food Restaurant,Creperie,Cuban Restaurant,Cupcake Shop,Deli / Bodega,Dessert Shop,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Empanada Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Food,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Gastropub,German Restaurant,Gluten-free Restaurant,Greek Restaurant,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Latin American Restaurant,Mac & Cheese Joint,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Molecular Gastronomy Restaurant,New American Restaurant,Noodle House,Pizza Place,Poke Place,Polish Restaurant,Poutine Place,Ramen Restaurant,Restaurant,Salad Place,Sandwich Place,Seafood Restaurant,Snack Place,Steakhouse,Sushi Restaurant,Taiwanese Restaurant,Tea Room,Thai Restaurant,Theme Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wings Joint
0,"Adelaide,King,Richmon",0.000000,0.033333,0.033333,0.000000,0.000000,0.050000,0.000000,0.000000,0.016667,0.033333,0.000000,0.000000,0.016667,0.016667,0.083333,0.000000,0.000000,0.000000,0.116667,0.016667,0.000000,0.000000,0.000000,0.000000,0.016667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.016667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.016667,0.000000,0.016667,0.000000,0.000000,0.016667,0.016667,0.000000,0.016667,0.000000,0.016667,0.000000,0.000000,0.000000,0.000000,0.016667,0.016667,0.033333,0.016667,0.000000,0.000000,0.016667,0.100000,0.033333,0.016667,0.016667,0.000000,0.033333,0.033333,0.000000,0.000000,0.050000,0.000000,0.033333,0.000000,0.000000,0.000000
1,Berczy Par,0.000000,0.000000,0.000000,0.038462,0.038462,0.076923,0.000000,0.038462,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.076923,0.000000,0.000000,0.000000,0.192308,0.000000,0.038462,0.038462,0.000000,0.000000,0.000000,0.000000,0.038462,0.000000,0.000000,0.000000,0.038462,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.038462,0.000000,0.000000,0.000000,0.000000,0.000000,0.038462,0.000000,0.000000,0.000000,0.038462,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.076923,0.000000,0.000000,0.076923,0.000000,0.038462,0.000000,0.000000,0.000000,0.038462,0.000000,0.038462,0.000000,0.000000,0.000000
2,"Brockton,Exhibition Place,Parkdale Villag",0.000000,0.000000,0.000000,0.000000,0.000000,0.090909,0.000000,0.000000,0.000000,0.181818,0.000000,0.000000,0.000000,0.090909,0.272727,0.000000,0.000000,0.000000,0.181818,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.090909,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.090909,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Business Reply Mail Processing Centre 969 Easter,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,

#### Let's confirm the new size

In [38]:
toronto_grouped.shape

(34, 82)

#### Let's print each neighborhood along with the top 5 most common venues

In [39]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    #print(temp)
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmon----
             venue  freq
0      Coffee Shop  0.12
1       Restaurant  0.10
2             Café  0.08
3           Bakery  0.05
4  Thai Restaurant  0.05


----Berczy Par----
                venue  freq
0         Coffee Shop  0.19
1          Restaurant  0.08
2                Café  0.08
3              Bakery  0.08
4  Seafood Restaurant  0.08


----Brockton,Exhibition Place,Parkdale Villag----
            venue  freq
0            Café  0.27
1     Coffee Shop  0.18
2  Breakfast Spot  0.18
3      Restaurant  0.09
4          Bakery  0.09


----Business Reply Mail Processing Centre 969 Easter----
                  venue  freq
0           Pizza Place   0.2
1         Burrito Place   0.2
2            Restaurant   0.2
3  Fast Food Restaurant   0.2
4               Brewery   0.2


----CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagar----
                             venue  freq
0                      Coffee Shop   1.0
1   

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [40]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [41]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']


# create columns according to number of top venues
columns = ['Neighborhood']


for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted9 = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted9['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted9.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted9.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmon",Coffee Shop,Restaurant,Café,Thai Restaurant,Bakery,Steakhouse,Salad Place,Breakfast Spot,Sushi Restaurant,Gastropub
1,Berczy Par,Coffee Shop,Restaurant,Seafood Restaurant,Café,Bakery,Comfort Food Restaurant,Japanese Restaurant,Diner,Greek Restaurant,Eastern European Restaurant
2,"Brockton,Exhibition Place,Parkdale Villag",Café,Coffee Shop,Breakfast Spot,Bakery,Burrito Place,Restaurant,Italian Restaurant,Ethiopian Restaurant,Empanada Restaurant,Eastern European Restaurant
3,Business Reply Mail Processing Centre 969 Easter,Brewery,Pizza Place,Fast Food Restaurant,Burrito Place,Restaurant,Doner Restaurant,Cuban Restaurant,Cupcake Shop,Deli / Bodega,Dessert Shop
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Coffee Shop,Comfort Food Restaurant,Creperie,Cuban Restaurant,Cupcake Shop,Deli / Bodega,Dessert Shop,Diner,Doner Restaurant,Donut Shop


<a id='item4'></a>

##  3. Cluster the Neighborhoods

Run *k*-means to cluster the neighborhood into 9 clusters.

In [42]:
# set number of clusters
kclusters = 9

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 3, 0, 6, 1, 6, 7, 6])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [46]:
# add clustering labels

neighborhoods_venues_sorted9.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = pd_Postal_codes_Toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted9.set_index('Neighborhood'), on='Neighborhood')

toronto_merged9 = toronto_merged.dropna()

#toronto_merged9=toronto_merged9.astype({'Cluster Labels':int})

toronto_merged9 #.head() # check the last columns!



,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
41,M4K,East Toronto,"The Danforth West,Riverdal",43.679557,-79.352188,6.0,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Bubble Tea Shop,Diner,Juice Bar,Pizza Place,Caribbean Restaurant,Café
42,M4L,East Toronto,"The Beaches West,India Bazaa",43.668999,-79.315572,3.0,Ice Cream Shop,Fast Food Restaurant,Fish & Chips Shop,Sushi Restaurant,Steakhouse,Pizza Place,Burrito Place,Sandwich Place,Italian Restaurant,Restaurant
43,M4M,East Toronto,Studio Distric,43.659526,-79.340923,1.0,Café,Coffee Shop,American Restaurant,Bakery,Gastropub,Brewery,Sandwich Place,Comfort Food Restaurant,Middle Eastern Restaurant,Italian Restaurant
45,M4P,Central Toronto,Davisville Nort,43.712751,-79.390197,2.0,Breakfast Spot,Sandwich Place,Dumpling Restaurant,Cuban Restaurant,Cupcake Shop,Deli / Bodega,Dessert Shop,Diner,Doner Restaurant,Donut Shop
46,M4R,Central Toronto,North Toronto Wes,43.715383,-79.405678,6.0,Coffee Shop,Diner,Chinese Restaurant,Fast Food Restaurant,Mexican Restaurant,Café,Restaurant,Dessert Shop,Cuban Restaurant,Cupcake Shop
47,M4S,Central Toronto,Davisvill,43.704324,-79.388790,6.0,Sandwich Place,Dessert Shop,Sushi Restaurant,Coffee Shop,Café,Italian Restaurant,Pizza Place,Seafood Restaurant,Greek Restaurant,Restaurant
49,M4V,Central Toronto,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",43.686412,-79.400049,1.0,Coffee Shop,Sushi Restaurant,Pizza Place,Burger Joint,Restaurant,Fried Chicken Joint,Vietnamese Restaurant,American Restaurant,Falafel Restaurant,Ethiopian Restaurant
51,M4X,Downtown Toronto,"Cabbagetown,St. James Tow",43.667967,-79.367675,6.0,Restaurant,Coffee Shop,Italian Restaurant,Chinese Restaurant,Café,Bakery,Pizza Place,Gastropub,Caribbean Restaurant,Indian Restaurant
52,M4Y,Downtown Toronto,Church and Wellesle,43.665860,-79.383160,6.0,Japanese Restaurant,Coffee Shop,Restaurant,Sushi Restaurant,Mediterranean Restaurant,Café,Gastropub,Juice Bar,Italian Restaurant,Indian Restaurant
53,M5A,Downtown Toronto,Harbourfron,43.654260,-79.360636,1.0,Coffee Shop,Mexican Restaurant,Breakfast Spot,Café,Restaurant,Bakery,Dessert Shop,Ice Cream Shop,French Restaurant,Asian Restaurant


Change the type of the "Cluster Labels"-column from Float to INT

In [47]:
toronto_merged9=toronto_merged9.astype({'Cluster Labels':int})


In [48]:
toronto_merged9.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
41,M4K,East Toronto,"The Danforth West,Riverdal",43.679557,-79.352188,6,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Bubble Tea Shop,Diner,Juice Bar,Pizza Place,Caribbean Restaurant,Café
42,M4L,East Toronto,"The Beaches West,India Bazaa",43.668999,-79.315572,3,Ice Cream Shop,Fast Food Restaurant,Fish & Chips Shop,Sushi Restaurant,Steakhouse,Pizza Place,Burrito Place,Sandwich Place,Italian Restaurant,Restaurant
43,M4M,East Toronto,Studio Distric,43.659526,-79.340923,1,Café,Coffee Shop,American Restaurant,Bakery,Gastropub,Brewery,Sandwich Place,Comfort Food Restaurant,Middle Eastern Restaurant,Italian Restaurant
45,M4P,Central Toronto,Davisville Nort,43.712751,-79.390197,2,Breakfast Spot,Sandwich Place,Dumpling Restaurant,Cuban Restaurant,Cupcake Shop,Deli / Bodega,Dessert Shop,Diner,Doner Restaurant,Donut Shop
46,M4R,Central Toronto,North Toronto Wes,43.715383,-79.405678,6,Coffee Shop,Diner,Chinese Restaurant,Fast Food Restaurant,Mexican Restaurant,Café,Restaurant,Dessert Shop,Cuban Restaurant,Cupcake Shop


Finally, let's visualize the resulting clusters

In [49]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]


# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged9['Latitude'], toronto_merged9['Longitude'], toronto_merged9['Neighborhood'], toronto_merged9['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='item5'></a>

## 4. Examine the Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

I first had 5 clusters, but most of the Neighborhoods were in 1 cluster which now constitutes Cluster 8 . So I have increased it to 9 to see if I can get some more variety.

#### Cluster 1 - CN Tower, Bathurst Quay, Island Ariport, harbour

(Reddish in the south) Airport, harbour.

In [50]:
toronto_merged9.loc[toronto_merged9['Cluster Labels'] == 0, toronto_merged9.columns[[2] + list(range(5, toronto_merged9.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
68,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0,Coffee Shop,Comfort Food Restaurant,Creperie,Cuban Restaurant,Cupcake Shop,Deli / Bodega,Dessert Shop,Diner,Doner Restaurant,Donut Shop


#### Cluster 2 - Coffee shops. mostly City centre, North-South


(Purple color North and south - mostly city centre 18 Neighborhoods


In [51]:
toronto_merged9.loc[toronto_merged9['Cluster Labels'] == 1, toronto_merged9.columns[[2] + list(range(5, toronto_merged9.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
43,Studio Distric,1,Café,Coffee Shop,American Restaurant,Bakery,Gastropub,Brewery,Sandwich Place,Comfort Food Restaurant,Middle Eastern Restaurant,Italian Restaurant
49,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",1,Coffee Shop,Sushi Restaurant,Pizza Place,Burger Joint,Restaurant,Fried Chicken Joint,Vietnamese Restaurant,American Restaurant,Falafel Restaurant,Ethiopian Restaurant
53,Harbourfron,1,Coffee Shop,Mexican Restaurant,Breakfast Spot,Café,Restaurant,Bakery,Dessert Shop,Ice Cream Shop,French Restaurant,Asian Restaurant
55,St. James Tow,1,Coffee Shop,Café,Restaurant,Italian Restaurant,Bakery,Diner,Breakfast Spot,Creperie,American Restaurant,Japanese Restaurant
56,Berczy Par,1,Coffee Shop,Restaurant,Seafood Restaurant,Café,Bakery,Comfort Food Restaurant,Japanese Restaurant,Diner,Greek Restaurant,Eastern European Restaurant
57,Central Bay Stree,1,Coffee Shop,Italian Restaurant,Burger Joint,Sandwich Place,Japanese Restaurant,Thai Restaurant,Bubble Tea Shop,Café,Salad Place,Ice Cream Shop
58,"Adelaide,King,Richmon",1,Coffee Shop,Restaurant,Café,Thai Restaurant,Bakery,Steakhouse,Salad Place,Breakfast Spot,Sushi Restaurant,Gastropub
59,"Harbourfront East,Toronto Islands,Union Statio",1,Coffee Shop,Italian Restaurant,Café,Restaurant,Brewery,Fried Chicken Joint,Pizza Place,Tea Room,Ice Cream Shop,Indian Restaurant
60,"Design Exchange,Toronto Dominion Centr",1,Coffee Shop,Café,Restaurant,Italian Restaurant,Seafood Restaurant,Gastropub,Japanese Restaurant,American Restaurant,Tea Room,Pizza Place
61,"Commerce Court,Victoria Hote",1,Coffee Shop,Restaurant,Café,American Restaurant,Gastropub,Italian Restaurant,Japanese Restaurant,Seafood Restaurant,Deli / Bodega,Bakery


#### Cluster 3 - Davisville North|

(Medium blue, North on the map)

Northemd Suburb "standard suburb" with a Dumpling restaurant and a donut shop. 

In [52]:
toronto_merged9.loc[toronto_merged9['Cluster Labels'] == 2, toronto_merged9.columns[[2] + list(range(5, toronto_merged9.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
45,Davisville Nort,2,Breakfast Spot,Sandwich Place,Dumpling Restaurant,Cuban Restaurant,Cupcake Shop,Deli / Bodega,Dessert Shop,Diner,Doner Restaurant,Donut Shop


#### Cluster 4 - Forest Hill North, Forest Hill West

(Light blue, in the middle of the map.) 

Northern Suburb - with Dumpling Restaurant and Donut Shop

In [53]:
toronto_merged9.loc[toronto_merged9['Cluster Labels'] == 3, toronto_merged9.columns[[2] + list(range(5, toronto_merged9.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
42,"The Beaches West,India Bazaa",3,Ice Cream Shop,Fast Food Restaurant,Fish & Chips Shop,Sushi Restaurant,Steakhouse,Pizza Place,Burrito Place,Sandwich Place,Italian Restaurant,Restaurant
87,Business Reply Mail Processing Centre 969 Easter,3,Brewery,Pizza Place,Fast Food Restaurant,Burrito Place,Restaurant,Doner Restaurant,Cuban Restaurant,Cupcake Shop,Deli / Bodega,Dessert Shop


#### Cluster 5 -  East-West


(Turcoise, from East to West, 8 Neighborhoods ) A mix of everything


In [54]:
toronto_merged9.loc[toronto_merged9['Cluster Labels'] == 4, toronto_merged9.columns[[2] + list(range(5, toronto_merged9.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
64,"Forest Hill North,Forest Hill Wes",4,Sushi Restaurant,Wings Joint,Dumpling Restaurant,Creperie,Cuban Restaurant,Cupcake Shop,Deli / Bodega,Dessert Shop,Diner,Doner Restaurant


#### Cluster 6 - Dowercourt Village, Dufferin The baker and the Brewer



(Light Turqouise, North West of the city) 


In [55]:
toronto_merged9.loc[toronto_merged9['Cluster Labels'] == 5, toronto_merged9.columns[[2] + list(range(5, toronto_merged9.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
76,"Dovercourt Village,Dufferi",5,Bakery,Brewery,Coffee Shop,Café,Middle Eastern Restaurant,Fast Food Restaurant,Doner Restaurant,Cupcake Shop,Food,Deli / Bodega


#### Cluster 7 - Business Reply Centre


In [56]:
toronto_merged9.loc[toronto_merged9['Cluster Labels'] == 6, toronto_merged9.columns[[2] + list(range(5, toronto_merged9.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
41,"The Danforth West,Riverdal",6,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Bubble Tea Shop,Diner,Juice Bar,Pizza Place,Caribbean Restaurant,Café
46,North Toronto Wes,6,Coffee Shop,Diner,Chinese Restaurant,Fast Food Restaurant,Mexican Restaurant,Café,Restaurant,Dessert Shop,Cuban Restaurant,Cupcake Shop
47,Davisvill,6,Sandwich Place,Dessert Shop,Sushi Restaurant,Coffee Shop,Café,Italian Restaurant,Pizza Place,Seafood Restaurant,Greek Restaurant,Restaurant
51,"Cabbagetown,St. James Tow",6,Restaurant,Coffee Shop,Italian Restaurant,Chinese Restaurant,Café,Bakery,Pizza Place,Gastropub,Caribbean Restaurant,Indian Restaurant
52,Church and Wellesle,6,Japanese Restaurant,Coffee Shop,Restaurant,Sushi Restaurant,Mediterranean Restaurant,Café,Gastropub,Juice Bar,Italian Restaurant,Indian Restaurant
54,"Ryerson,Garden Distric",6,Coffee Shop,Middle Eastern Restaurant,Bubble Tea Shop,Japanese Restaurant,Café,Pizza Place,Ramen Restaurant,Restaurant,Fast Food Restaurant,Diner
67,"Chinatown,Grange Park,Kensington Marke",6,Vietnamese Restaurant,Café,Vegetarian / Vegan Restaurant,Coffee Shop,Dumpling Restaurant,Mexican Restaurant,Dessert Shop,Burger Joint,Chinese Restaurant,Bakery
77,"Little Portugal,Trinit",6,Asian Restaurant,Coffee Shop,Restaurant,Vietnamese Restaurant,Pizza Place,Café,Deli / Bodega,Mac & Cheese Joint,Japanese Restaurant,Italian Restaurant
83,"Parkdale,Roncesvalle",6,Coffee Shop,Eastern European Restaurant,Cuban Restaurant,Dessert Shop,Restaurant,Italian Restaurant,Breakfast Spot,Empanada Restaurant,Ethiopian Restaurant,Falafel Restaurant
84,"Runnymede,Swanse",6,Coffee Shop,Café,Sushi Restaurant,Pizza Place,Italian Restaurant,Restaurant,Gastropub,Juice Bar,Falafel Restaurant,Burrito Place


#### Cluster 8 - High Park, The Junction South

(Faded Orange - to the West of the city) - 4 "proper" restaurants


In [57]:
toronto_merged9.loc[toronto_merged9['Cluster Labels'] == 7, toronto_merged9.columns[[2] + list(range(5, toronto_merged9.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
66,"Harbord,University of Toront",7,Café,Japanese Restaurant,Bakery,Restaurant,Chinese Restaurant,Comfort Food Restaurant,Noodle House,Dessert Shop,Italian Restaurant,Sandwich Place
75,Christi,7,Café,Coffee Shop,Restaurant,Diner,Italian Restaurant,Donut Shop,Cuban Restaurant,Cupcake Shop,Deli / Bodega,Dessert Shop


#### Cluster 9 - Cafe club
Young Family


Burnt orange - 3 Neighborhhooods )
Inbetween the City Centre Neighborhoods is Christie. It does not look at all like the others with a grocery store in first position, a Candy store in 4th and a baby store in 6th. Add a nightclub to the mix. Young Family?

In [58]:
toronto_merged9.loc[toronto_merged9['Cluster Labels'] == 8, toronto_merged9.columns[[2] + list(range(5, toronto_merged9.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
82,"High Park,The Junction Sout",8,Mexican Restaurant,Thai Restaurant,Café,Fried Chicken Joint,Fast Food Restaurant,Gastropub,Bakery,Cajun / Creole Restaurant,Italian Restaurant,Diner


## 5. Analysis


Two things stand out;
    • The north to south grouping of purple-coloured neighborhoods with 18 establisments (cluster 2)
    • The “wave” of turquise-coloured neighborhoods running east to west with 8 establishments.(Cluster 5) with a good mixture of food establishments

The next cluster is the Cafe-club, with 3 establishments (Cluster 8) Probably because it is around the University.

Then there are 2 clusters with all-together 3 Neighborhoods – Clusters 3 and 4. that contains Dumpling Restaurants, Donner and Donut Shop

Cluster 8  ( High Park, The Junction South, west of the city) have 4 "proper" restaurants – Mexican, Thai, Cajun/Creole and Italian. 

Cluster 6 (Dowercourt Village, Dufferin) has a bakery and a brewery as the star attractions.
Cluster 1 consist of the Neighbourhoods  CN Tower, Bathurst Quay, Island Ariport, harbour. I guess mostly “on the run” meals. But I can be completely wrong.

Lastly Cluster 7 – contains a “neighborhood” “Business Reply Centre”. It is a post code used for mass-mailing. 


To Conclude; I will think twice before opening a coffee shop in the North-South groiping of neighborhoods. The "wave" of Torquise-coloured neighborhoods might be a good environment to open a food-establishment as there are a great variety of establishments in the 8 Neighbourhoods from east to west. 


### References

This notebook is part of a course on **Coursera** called *Applied Data Science Capstone*. If you accessed this notebook outside the course, you can take this course online by clicking [here](http://cocl.us/DP0701EN_Coursera_Week3_LAB2).

<hr>

Copyright &copy; 2018 [Cognitive Class](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).